In [1]:
import logging
import os
from pathlib import Path
import sqlite3 as sql

from dash import Dash, html, dash_table, dcc, callback, Output, Input
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc


In [2]:
external_stylesheets = [dbc.themes.JOURNAL]
app = Dash(__name__, external_stylesheets=external_stylesheets)

In [3]:
output_folder = Path(os.getenv('GRADDNODI_OUTPUT', 'Output/'))
print(list(Path('/media/idris/PhD').glob('*')))
print(output_folder)
print(list(output_folder.glob('*')))
output_options = [str(dir.parts[-1]) for dir in output_folder.glob('*') if dir.is_dir()]
print(output_options)

[PosixPath('/media/idris/PhD/.Trash-1000'), PosixPath('/media/idris/PhD/idris'), PosixPath('/media/idris/PhD/Graddnodi'), PosixPath('/media/idris/PhD/lost+found')]
/media/idris/PhD/Graddnodi/Output
[PosixPath('/media/idris/PhD/Graddnodi/Output/Test'), PosixPath('/media/idris/PhD/Graddnodi/Output/Default'), PosixPath('/media/idris/PhD/Graddnodi/Output/Bushy Park Study'), PosixPath('/media/idris/PhD/Graddnodi/Output/Test2')]
['Test', 'Default', 'Bushy Park Study', 'Test2']


In [4]:
@callback(
    Output('folder-path', 'data'),
    Input('folder-name', 'value')
)
def folder_path(name):
    return str(output_folder.joinpath(name))


@callback(
    Output('folder-output-text', 'children'),
    Input('folder-path', 'data') 
)
def folder_path_text(path):
    return f'Using data from {path}'


@callback(
    Output('db-index', 'data'),
    Input('folder-path', 'data')
    )
def get_df_index(path):
    con = sql.connect(Path(path).joinpath('Results').joinpath('Results.db'))
    raw_index = pd.read_sql(
        sql='SELECT DISTINCT "Reference", "Field", "Calibrated", "Technique", "Scaling Method", "Variables" FROM Results;',
        con=con
    )
    con.close()
    return raw_index.to_json(orient='split')


@callback(
    Output('reference-options', 'options'),
    Input('db-index', 'data')
)
def ref_opts(data):
    df = pd.read_json(data, orient='split')
    return [
        {'label': i, 'value': i} for i in df['Reference'].unique()
    ]


@callback(
    Output('field-options', 'options'),
    Output('calibrated-device-options', 'options'),
    Output('technique-options', 'options'),
    Output('scaling-options', 'options'),
    Output('var-options', 'options'),
    Output('chosen-combo-index', 'data'),
    #Output('combination-table', 'figure'),
    Output('num-of-runs', 'children'),
    Input('db-index', 'data'),
    Input('reference-options', 'value'),
    Input('field-options', 'value'),
    Input('calibrated-device-options', 'value'),
    Input('technique-options', 'value'),
    Input('scaling-options', 'value'),
    Input('var-options', 'value'),
)
def filter_options(data, ref_d, fields, cal_d, tech, sca, var):
    levels = {
        "Field": fields,
        "Calibrated": cal_d,
        "Technique": tech,
        "Scaling Method": sca,
        "Variables": var
    }
    db_index = pd.read_json(data, orient='split')
    df = db_index[db_index['Reference'] == ref_d]
    s_df = df.copy(deep=True)
    for name, col in levels.items():
        if not col:
            cols = s_df[name].unique()
        else:
            cols = col
        s_df = s_df[s_df[name].isin(cols)]
    
    return (
        [{'label': i, 'value': i} for i in sorted(df['Field'].unique())],
        [{'label': i, 'value': i} for i in sorted(df['Calibrated'].unique())],
        [{'label': i.replace(' Regression', ''), 'value': i} for i in sorted(df['Technique'].unique())],
        [{'label': i, 'value': i} for i in sorted(df['Scaling Method'].unique())],
        [{'label': i, 'value': i} for i in sorted(df['Variables'].unique())],
        s_df.to_json(orient='split'),
        #table_fig,
        f'{s_df.shape[0]} combinations'
    )

@callback(
    #Output('results-df', 'data'),
    Output('results-table', 'figure'),
    Input('chosen-combo-index', 'data'),
    Input('folder-path', 'data'),
    Input('reference-options', 'value'),
    )
def get_results_df(data, path, ref_d):
    if not ref_d:
        return go.Figure(
            data=[
                go.Table(
                    header={
                    },
                    cells={
                    }
                )
            ]
        )
    df = pd.read_json(data, orient='split')
    query_list = [
        "SELECT *",
        "FROM Results"
    ]
    for i, (name, vals) in enumerate(df.items()):
        val_list = "', '".join(vals.unique())
        query_list.append(f'''{"WHERE" if i == 0 else "AND"} "{name}" in ('{val_list}')''')
    con = sql.connect(Path(path).joinpath('Results').joinpath('Results.db'))
    query="\n".join(query_list)
    sql_data = pd.read_sql(
        sql=f'{query};',
        con=con
    )
    con.close()

    table_fig = go.Figure(
        data=[
            go.Table(
                header={
                    'values': list(sql_data.columns),
                    'align': 'left'
                },
                cells={
                    'values': sql_data.transpose().values.tolist(),
                    'align': 'left'
                }
            )
        ]
    )
    
    return table_fig
        #sql_data.to_json(orient='split'),
        

In [5]:
item_stores = [
    dcc.Store(id='folder-path'),
    dcc.Store(id='db-index'),
    dcc.Store(id='results-df'),
    dcc.Store(id='chosen-combo-index')
]

top_row = [
    dbc.Row([html.Div('Graddnodi', className='h1', style={'text-align': 'center'})]),
    dbc.Row(
        [
            dbc.Col(
                [
                    dcc.Dropdown(
                        sorted(output_options),
                        sorted(output_options)[0],
                        id='folder-name'
                    ),
                ]
            ),
            dbc.Col(
                [
                    html.Div(id='folder-output-text')
                ]
            )
        ]
    )
    
]

checklist_options = {
    "overflow-y": "scroll",
    "height": "20vh"
}

selections = [
    dbc.Row(
        [
            dbc.Col(
                [
                    dcc.RadioItems(id='reference-options', style=checklist_options)        
                ]
            ),
            
            dbc.Col(
                [
                    dcc.Checklist(id='field-options', style=checklist_options)        
                ]
            ),
            dbc.Col(
                [
                    dcc.Checklist(id='calibrated-device-options', style=checklist_options)        
                ]
            ),
            dbc.Col(
                [
                    dcc.Checklist(id='technique-options', style=checklist_options)        
                ]
            ),
            dbc.Col(
                [
                    dcc.Checklist(id='scaling-options', style=checklist_options)        
                ]
            ),
            dbc.Col(
                [
                    dcc.Checklist(id='var-options', style=checklist_options)        
                ]
            ),
            
        ]
    )
]

results_table = [
    dbc.Row([html.Div('Results', className='h2', style={'text-align': 'center'})]),
    dbc.Row(
                [
                    dcc.Graph(figure={}, id='results-table')
                ],
            ),
    dbc.Row([html.Div(id='num-of-runs', style={'text-align': 'center'})]),
]


app.layout = dbc.Container(
    [        
        *item_stores,
        *top_row,
        html.Hr(),
        *selections,
        html.Hr(),
        *results_table,
        html.Hr(),
        html.Div(id='row-test')
    ]
)


In [6]:
app.run(jupyter_mode="external", debug=True)

Dash app running on http://127.0.0.1:8050/
